In [1]:
import torch.nn as nn
import torch
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import scipy.stats as stats

In [2]:
fluence = pd.read_csv('reinserted_thermal_fluence.csv')

In [3]:
fluence

,# timestamp,cesium_val_0,cesium_val_1,cesium_val_2,cesium_val_3,cesium_val_4,cesium_val_5,cesium_val_6,cesium_val_7,cesium_val_8,...,fluence_val_803,fluence_val_804,fluence_val_805,fluence_val_806,fluence_val_807,fluence_val_808,fluence_val_809,fluence_val_810,fluence_val_811,fluence_val_812
0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.0,3.474796e-08,1.707146e-07,7.250270e-07,1.518698e-07,2.655534e-07,1.096870e-07,4.931048e-07,5.913665e-08,7.028061e-07,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3.0,2.246938e-06,2.872698e-06,2.737213e-06,3.099643e-06,2.319968e-06,2.971253e-06,4.813968e-07,1.661822e-06,3.443706e-06,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.0,4.538420e-06,4.424400e-06,3.635429e-06,5.124641e-06,4.557323e-06,5.301739e-06,3.826000e-06,6.284903e-06,5.364588e-06,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,191.0,0.000000e+00,5.054584e-05,5.790360e-05,4.756137e-05,0.000000e+00,0.000000e+00,6.990133e-05,0.000000e+00,4.825021e-05,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
192,192.0,5.011145e-05,8.686642e-05,0.000000e+00,4.515687e-05,0.000000e+00,0.000000e+00,4.304290e-05,4.031293e-05,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193,193.0,0.000000e+00,0.000000e+00,4.521334e-05,7.796768e-05,0.000000e+00,4.886403e-05,0.000000e+00,0.000000e+00,6.935493e-05,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
194,194.0,0.000000e+00,4.109691e-05,0.000000e+00,9.213985e-05,7.296321e-05,0.000000e+00,0.000000e+00,3.900867e-05,4.373237e-05,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
cesium_only = fluence.iloc[:,:814]
cesium_only.head()

,# timestamp,cesium_val_0,cesium_val_1,cesium_val_2,cesium_val_3,cesium_val_4,cesium_val_5,cesium_val_6,cesium_val_7,cesium_val_8,...,cesium_val_803,cesium_val_804,cesium_val_805,cesium_val_806,cesium_val_807,cesium_val_808,cesium_val_809,cesium_val_810,cesium_val_811,final_cesium_val
0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,3.474796e-08,1.707146e-07,7.250270e-07,1.518698e-07,2.655534e-07,1.096870e-07,4.931048e-07,5.913665e-08,7.028061e-07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3.0,2.246938e-06,2.872698e-06,2.737213e-06,3.099643e-06,2.319968e-06,2.971253e-06,4.813968e-07,1.661822e-06,3.443706e-06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.0,4.538420e-06,4.424400e-06,3.635429e-06,5.124641e-06,4.557323e-06,5.301739e-06,3.826000e-06,6.284903e-06,5.364588e-06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
fluence_only = fluence.iloc[:,814:]

In [18]:
timestamp = fluence[['# timestamp']]

In [19]:
fluence_t = pd.concat([timestamp, fluence_only], axis=1)
fluence_t

,# timestamp,fluence_val_0,fluence_val_1,fluence_val_2,fluence_val_3,fluence_val_4,fluence_val_5,fluence_val_6,fluence_val_7,fluence_val_8,...,fluence_val_803,fluence_val_804,fluence_val_805,fluence_val_806,fluence_val_807,fluence_val_808,fluence_val_809,fluence_val_810,fluence_val_811,fluence_val_812
0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.0,2.713469e+17,1.456545e+18,5.577840e+18,1.790125e+18,2.542165e+18,9.863451e+17,2.894167e+18,5.488056e+17,6.859838e+18,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3.0,2.110015e+19,2.899520e+19,2.441440e+19,3.120383e+19,2.212393e+19,2.798426e+19,4.244877e+18,1.414141e+19,3.368271e+19,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.0,4.484809e+19,4.345077e+19,3.658466e+19,4.824004e+19,4.459397e+19,5.148963e+19,3.355578e+19,5.906011e+19,5.683764e+19,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,191.0,0.000000e+00,5.490824e+20,6.347695e+20,5.162419e+20,0.000000e+00,0.000000e+00,7.961712e+20,0.000000e+00,5.162309e+20,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
192,192.0,5.513913e+20,1.059060e+21,0.000000e+00,5.010226e+20,0.000000e+00,0.000000e+00,4.742642e+20,4.327103e+20,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193,193.0,0.000000e+00,0.000000e+00,4.952903e+20,8.838849e+20,0.000000e+00,5.269729e+20,0.000000e+00,0.000000e+00,7.984504e+20,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
194,194.0,0.000000e+00,4.588275e+20,0.000000e+00,1.125791e+21,8.312452e+20,0.000000e+00,0.000000e+00,4.315316e+20,4.846104e+20,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
counts, bin_edges = np.histogram(fluence_t.iloc[100])

In [23]:
counts

array([357,   0,   2, 163,  54,  34,  77,  62,  38,  27])

In [24]:
bin_edges

array([0.00000000e+00, 1.41239125e+20, 2.82478250e+20, 4.23717375e+20,
       5.64956500e+20, 7.06195624e+20, 8.47434749e+20, 9.88673874e+20,
       1.12991300e+21, 1.27115212e+21, 1.41239125e+21])

In [113]:
fluence_t.iloc[:,1:].mean().mean()

4.369526669891298e+20

In [114]:
1.4412777313825236e+21/150

9.608518209216823e+18

In [115]:
fluence_t.iloc[:,:-1].min().min()

0.0

In [116]:
fluence_t=fluence_t.set_index('# timestamp')
row_means = fluence_t.mean(axis=1)
fluence_t = fluence_t.reset_index()

In [117]:
# Read data
Tbl = fluence_t
 
# Extract data
Xvals = Tbl.iloc[:,1:-1].to_numpy()
Yvals = Tbl.iloc[:,-1].to_numpy()
timestamps = Tbl.iloc[:,0].to_numpy()
nRows, nCols = Xvals.shape
#Xavg = np.mean(Xvals, axis=1)
 
# define bins
binMax = 1.4412777313825237e+21
binInt = binMax/200
hEdges = np.arange(0, binMax+binInt, binInt)
nBins = len(hEdges)-1
 
# fill histogram matrix
nCountsX = np.zeros((nRows, nBins), dtype=np.uint16)
for n in range(nRows):
    nCountsX[n,:], _ = np.histogram(Xvals[n,:], hEdges)
    
print(nRows, nBins)

196 201


In [118]:
counts_df = pd.DataFrame(nCountsX)

In [119]:
counts_df

,0,1,2,3,4,5,6,7,8,9,...,191,192,193,194,195,196,197,198,199,200
0,812,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,812,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,778,34,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,102,161,130,198,200,21,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,104,0,0,1,67,127,114,178,156,60,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,355,0,0,0,0,0,0,0,0,0,...,3,1,1,2,0,0,0,0,0,0
192,347,0,0,0,0,0,0,0,0,0,...,1,0,1,1,2,0,0,0,0,0
193,344,0,0,0,0,0,0,0,0,0,...,2,0,0,1,2,0,0,0,0,0
194,329,0,0,0,0,0,0,0,0,0,...,2,5,4,2,0,0,1,0,0,0


In [120]:
def bin_names(arr):
    arr_names = []
    for i in range(len(arr)):
        arr[i] = np.format_float_scientific(arr[i], unique=False, precision=2)
        if i+2 > len(arr):
            break
        else:
            arr[i+1] = np.format_float_scientific(arr[i+1], unique=False, precision=2)
            arr_names.append(str([str(arr[i]) + ':' + str(arr[i+1])]))
    return arr_names

hEdges_name = bin_names(hEdges)


In [121]:
timestamp = fluence['# timestamp'].values
counts_df.index = timestamps
counts_df.index.name = 't'
counts_df.columns = hEdges_name
counts_df['means'] = row_means

In [122]:
counts_df.shape

(196, 202)

In [125]:
counts_df.head()

,['0.0:7.21e+18'],['7.21e+18:1.44e+19'],['1.44e+19:2.16e+19'],['2.16e+19:2.88e+19'],['2.88e+19:3.6e+19'],['3.6e+19:4.32e+19'],['4.32e+19:5.04e+19'],['5.04e+19:5.77e+19'],['5.77e+19:6.49e+19'],['6.49e+19:7.21e+19'],...,['1.38e+21:1.39e+21'],['1.39e+21:1.4e+21'],['1.4e+21:1.41e+21'],['1.41e+21:1.41e+21'],['1.41e+21:1.42e+21'],['1.42e+21:1.43e+21'],['1.43e+21:1.43e+21'],['1.43e+21:1.44e+21'],['1.44e+21:1.45e+21'],means
t,,,,,,,,,,,,,,,,,,,,,
0.0,812,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000e+00
1.0,812,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000e+00
2.0,778,34,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2.103589e+18
3.0,102,161,130,198,200,21,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2.049087e+19
4.0,104,0,0,1,67,127,114,178,156,60,...,0,0,0,0,0,0,0,0,0,4.457204e+19


In [126]:
counts_df.to_csv('fluence.csv')

In [ ]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error

In [ ]:
# define the dataset classes
from sklearn.preprocessing import StandardScaler, MinMaxScaler


class ReactorData(Dataset):
    def __init__(self,file_name, sequence_length, start_percent = 0, end_percent = 1):
        data = pd.read_csv(file_name, skiprows=[0]) # skip the first line
        
        length = data.shape[0]
        data = data[ int(length * start_percent)  : int(length * end_percent)]
        
        # print(data.shape)
        
        
        self.labels = data.iloc[:, -1:]
        self.data = data.iloc[:, 1:-1] # skip the first time column
        
        mm = MinMaxScaler()
        ss = StandardScaler()


        self.data = ss.fit_transform(self.data)
        self.labels = mm.fit_transform(self.labels) 
        
        self.sequence_length = sequence_length
        
    def __len__(self):
        return len(self.labels)//self.sequence_length
    
    def __getitem__(self,idx):
        idx = idx * self.sequence_length
        
        return (torch.tensor(self.data[idx : idx+ self.sequence_length])).double(), \
    (torch.tensor(self.labels[idx : idx+ self.sequence_length])).double()

        